<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Анализ данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Общий-выводы" data-toc-modified-id="Общий-выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Анализ данных

Импортируем необходимые библиотеки

In [103]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier

from tqdm import notebook

Загрузим наш датасет и посмотрим на него

In [55]:
df = pd.read_csv('./toxic_comments.csv')

In [56]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [57]:
print('Количество строк в датасете равно {} шт'.format(len(df)))

Количество строк в датасете равно 159571 шт


Посмотрим на соотношения классов

In [58]:
pd.DataFrame({'Количество':df['toxic'].value_counts(), 
              '% от количества': (df['toxic'].value_counts()/len(df)*100).round(2),
             'Тип комментария': ['Позитивные', 'Негативные']})

,Количество,% от количества,Тип комментария
0,143346,89.83,Позитивные
1,16225,10.17,Негативные


Посмотрим на количество пропусков

In [59]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [60]:
print('Количество дубликатов в данных равно {}'.format(df.duplicated().sum()))

Количество дубликатов в данных равно 0


### Вывод

Мы загрузили и посмотрели на данные. В нашем датасете 159571 строк содержащих комментарии пользователей, из них:
- 143346 шт являются 'Позитивными', это почти 90% данных
- 16225 шт являются 'Негативными', это около 10% данных  

В данных нет пропусков и дубликатов. Можно приступать к обработке данных

## Подготовка данных

С помощью регулярных выражений очистим тест от лишней информации

In [61]:
df['text_cleared'] = df['text'].apply(lambda x: re.sub('[^A-Za-z]', ' ', x.lower()))

In [62]:
df.head()

,text,toxic,text_cleared
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m s...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on im...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember...


Загрузим список стоп-слов из английского языка. Далее отфильтруем наши сообщения, возьмем слова которые не находятся в списке стоп слов

In [63]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/maxim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
df['text_split'] = df['text_cleared'].map(lambda x: x.split())

In [65]:
df['not_stopwords'] = df['text_split'].map(lambda x: [word for word in x if word not in stop_words])

In [66]:
df.head()

,text,toxic,text_cleared,text_split,not_stopwords
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, made, under, my...","[explanation, edits, made, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m s...,"[d, aww, he, matches, this, background, colour...","[aww, matches, background, colour, seemingly, ..."
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it...,"[hey, man, i, m, really, not, trying, to, edit...","[hey, man, really, trying, edit, war, guy, con..."
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on im...,"[more, i, can, t, make, any, real, suggestions...","[make, real, suggestions, improvement, wondere..."
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember...,"[you, sir, are, my, hero, any, chance, you, re...","[sir, hero, chance, remember, page]"


Получим леммы полученных предложений

In [67]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /Users/maxim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/maxim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
        
    return lemmatized_output

In [69]:
df['lemmas'] = df['not_stopwords'].apply(lambda x: [lemmatize(" ".join(x))])

In [70]:
X = df['lemmas']
y = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

print('Количество элементов обучающей выборки: {:.0f}%'.format(X_train.shape[0]/df.shape[0] * 100))
print('Количество элементов тестовой выборки: {:.0f}%'.format(X_test.shape[0]/df.shape[0] * 100))

Количество элементов обучающей выборки: 80%
Количество элементов тестовой выборки: 20%


Вычислим tf-idf для текстов

In [71]:
X_train = X_train.astype('U')
X_test = X_test.astype('U')

### Вывод

На данном этапе мы получили леммы в предложениях. Далее разбили данные на обучающую и тестовую выборку, теперь можно приступать к обучению моделей

## Обучение

Построим следующие модели:

- Логистическая регрессия
- Дерево решений
- Случайый лес

In [85]:
results = []

### Логистическая регрессия

In [81]:
def log_reg(model_name, X_train, y_train, X_test, y_test):
    
    pipe_lr = Pipeline([('tfidf', TfidfVectorizer(stop_words=stop_words)),
                        ('clf', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=123))])
    
    grid_params_lr = [{'clf__fit_intercept': [True, False],
                       'clf__C': [0.1, 1, 10, 100],
                       'clf__solver': ['liblinear', 'lbfgs']}]
    
    lr_model = GridSearchCV(estimator=pipe_lr,
                      param_grid=grid_params_lr,
                      scoring='f1',
                      cv=3,
                      n_jobs=-1)
    
    lr_model.fit(X_train, y_train)
    
    predictions = lr_model.predict(X_test)
    
    f1 = f1_score(y_test, predictions)
    
    return {
        'Название модели': model_name,
        'Параметры модели': lr_model.best_params_,
        'f1_score': f1
    }

In [82]:
log_reg_model = log_reg('LogisticRegression', X_train, y_train, X_test, y_test)

In [86]:
results.append(log_reg_model)

In [87]:
pd.DataFrame(results)

,Название модели,Параметры модели,f1_score
0,LogisticRegression,"{'clf__C': 10, 'clf__fit_intercept': True, 'cl...",0.76207


### Дерево решений

In [93]:
def dt_classifier(model_name, X_train, y_train, X_test, y_test):
    
    pipe_dt = Pipeline([('tfidf', TfidfVectorizer(stop_words=stop_words)),
                        ('clf', DecisionTreeClassifier(class_weight='balanced', random_state=123))])
    
    grid_params_dt = [{'clf__max_depth':[5, 10, 15],
                       'clf__min_samples_split':[5, 7, 9],
                       'clf__min_samples_leaf':[3, 5, 7],
                       'clf__criterion': ['gini', 'entropy']}]
    
    dt_model = GridSearchCV(estimator=pipe_dt,
                      param_grid=grid_params_dt,
                      scoring='f1',
                      cv=3,
                      n_jobs=-1)
    
    dt_model.fit(X_train, y_train)
    
    predictions = dt_model.predict(X_test)
    
    f1 = f1_score(y_test, predictions)
    
    return {
        'Название модели': model_name,
        'Параметры модели': dt_model.best_params_,
        'f1_score': f1
    }

In [94]:
dt_classifier_model = dt_classifier('DecisionTreeClassifier', X_train, y_train, X_test, y_test)

In [95]:
results.append(dt_classifier_model)

In [96]:
pd.DataFrame(results)

,Название модели,Параметры модели,f1_score
0,LogisticRegression,"{'clf__C': 10, 'clf__fit_intercept': True, 'cl...",0.762070
1,DecisionTreeClassifier,"{'clf__criterion': 'entropy', 'clf__max_depth'...",0.612802


### Случайный лес

In [98]:
def rf_classifier(model_name, X_train, y_train, X_test, y_test):
    
    pipe_rf = Pipeline([('tfidf', TfidfVectorizer(stop_words=stop_words)),
                        ('clf', RandomForestClassifier(class_weight='balanced', random_state=123))])
    
    grid_params_rf = [{'clf__max_depth':[10, 15, 20, 25, 30],
                       'clf__n_estimators':[30, 40, 50],
                       'clf__max_features': ['auto', 'sqrt', 'log2'],
                       'clf__criterion': ['gini', 'entropy']}]

    rf_model = GridSearchCV(estimator=pipe_rf,
                      param_grid=grid_params_rf,
                      scoring='f1',
                      cv=3,
                      n_jobs=-1)
    
    rf_model.fit(X_train, y_train)
    
    predictions = rf_model.predict(X_test)
    
    f1 = f1_score(y_test, predictions)
    
    return {
        'Название модели': model_name,
        'Параметры модели': rf_model.best_params_,
        'f1_score': f1
    }

In [100]:
rf_classifier_model = rf_classifier('RandomForestClassifier', X_train, y_train, X_test, y_test)

In [101]:
results.append(rf_classifier_model)

In [102]:
pd.DataFrame(results)

,Название модели,Параметры модели,f1_score
0,LogisticRegression,"{'clf__C': 10, 'clf__fit_intercept': True, 'cl...",0.762070
1,DecisionTreeClassifier,"{'clf__criterion': 'entropy', 'clf__max_depth'...",0.612802
2,RandomForestClassifier,"{'clf__criterion': 'gini', 'clf__max_depth': 3...",0.396576


### Вывод

На данном этапе мы обучили 3 модели и получили следующие результаты:

- **Логистическая регрессия** получила результат **f1_score** на тестовой выборке равный 0.762070
- **Дерево решений** получило результат **f1_score** на тестовой выборке равный 0.612802
- **Случайый лес** получила результат **f1_score** на тестовой выборке равный 0.396576

**Логистическая регрессия** показала лучший результат

## Общий выводы

При выполнении работы, мы загрузили и посмотрели на данные. В нашем датасете 159571 строк содержащих комментарии пользователей, из них:
- 143346 шт являются **'Позитивными'**, это почти 90% данных
- 16225 шт являются **'Негативными'**, это около 10% данных

Убедились, что в данных нет пропусков и дубликатов. 

Далее приступили к обработке данных. На данном этапе мы получили леммы в предложениях и разбили данные на обучающую и тестовую выборку, после этого приступили к обучению моделей.  

Мы обучили 3 модели и получили следующие результаты:
- **Логистическая регрессия** получила результат **f1_score** на тестовой выборке равный 0.762070
- **Дерево решений** получило результат **f1_score** на тестовой выборке равный 0.612802
- **Случайый лес** получила результат **f1_score** на тестовой выборке равный 0.396576

**Логистическая регрессия** показала лучший результат